In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [3]:
def get_emojis():
    emojis_folder = 'C:\\Users\\Saatwik Bisaria\\\\hand_emo\\'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [5]:
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

In [7]:
def blend_transparent(face_img, overlay_t_img):
    overlay_img = overlay_t_img[:,:,:3] 
    overlay_mask = overlay_t_img[:,:,3:]  
    background_mask = 255 - overlay_mask
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))
    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [8]:
model = load_model('emojinator_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [9]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours,hierachy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

In [10]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
4 0.5482926
4 0.8466688
4 0.8062027
4 0.8840836
4 0.6759937
4 0.76309514
5 0.9999428
5 0.9831198
5 0.57027745
5 0.83220565
7 0.7166199
7 0.7853668
7 0.81066257
7 0.9414114
5 0.5840085
5 0.55332714
7 0.49854353
7 0.93303263
7 0.9328887
7 0.86067754
11 0.8458554
11 0.83796215
11 0.7895204
11 0.5379287
11 0.5847274
5 0.8200532
11 0.6587024
5 0.96218646
5 0.9281796
5 0.7886315
5 0.8069156
5 0.95248157
5 0.93298167
5 0.7754614
5 0.99865603
5 0.9982198
5 0.99830073
5 0.9035839
5 0.95773286
5 0.9977423
5 0.99992526
5 0.9294769
2 0.50374615
5 0.9764872
5 0.9998258
5 0.99738973
5 0.4206948
4 0.15378909
5 0.4201944
3 0.4358925
5 0.99951744
5 0.9999422
5 0.9998486
5 0.99942255
5 0.9984962
5 0.9983071
5 0.998637
5 0.9979747
5 0.99901354
5 0.99841344
5 0.99809176
5 0.9951946
5 0.8166523
5 0.78333634
11 0.97215354
11 0.9459328
11 0.65952814
11 0.98779243
11 0.77390224
11 0.7299005
11 0.9850043
11 0.9868867
11 0.9964831
11 0.9677059
11 0.998596
11 0.99956375
11 0.99334

11 0.7830803
3 0.48715034
9 0.3095177
11 0.81427205
3 0.39631543
11 0.3614931
11 0.3614931
9 0.57200956
11 0.36461154
11 0.85232526
9 0.5798325
11 0.8796468
11 0.7151008
11 0.46939337
11 0.985797
3 0.36814934
9 0.25066283
11 0.8936225
11 0.8936225
9 0.3300663
11 0.7544685
11 0.37878639
3 0.5123265
11 0.35924807
11 0.67455196
9 0.26940244
9 0.3308377
11 0.70307344
11 0.9036963
11 0.5944759
3 0.2347975
3 0.2347975
11 0.43740112
11 0.5923442
5 0.3170649
11 0.92871004
11 0.94605017
11 0.8996896
5 0.9894728
5 0.95719606
1 0.52008975
1 0.9715883
1 0.998198
1 0.998198
1 0.9993436
1 0.99993634
1 0.99995184
1 0.9999901
1 0.999987
1 0.9999356
1 0.9993474
1 0.99918514
1 0.99976844
1 0.99953544
1 0.9976648
1 0.9976648
1 0.9940783
1 0.98779535
1 0.9817871
1 0.98412055
1 0.95400274
1 0.99704117
1 0.991892
1 0.98869765
1 0.9988223
1 0.99119616
1 0.9981115
1 0.9981115
1 0.99022174
1 0.9943626
1 0.9740389
1 0.98358214
1 0.991422
1 0.9925056
1 0.9945425
1 0.9918542
1 0.9995585
1 0.9992029
1 0.998047
1 0

1 0.99492466
1 0.99987125
1 0.96124935
1 0.96680963
1 0.99999344
1 0.6635895
1 0.8643649
8 0.9478719
8 0.98278195
8 0.99936694
8 0.91882885
8 0.99813384
2 0.99540126
8 0.33482277
5 0.5092951
8 0.81607217
2 0.9429456
8 0.9346425
8 0.9991053
8 0.99856997
8 0.99870706
2 0.95354414
8 0.9726563
8 0.9962948
8 0.9949798
8 0.99880385
8 0.9989611
2 0.59469664
8 0.6200312
2 0.94727767
2 0.46611914
2 0.7186738
2 0.35539186
7 0.68561834
5 0.9723332
5 0.9992894
5 0.9997155
5 0.9999274
5 0.999962
5 0.99999213
5 0.99999523
5 0.9999702
5 0.9999807
5 0.99997365
5 0.99997544
5 0.9999924
5 0.99998915
5 0.9814014
5 1.0
7 0.9989735
7 0.811411
5 0.9082149
7 0.9233045
11 0.65395963
7 0.91942966
7 0.58680373
3 0.92822164
6 0.6430169
6 0.7887002
6 0.999295
6 1.0
6 0.9984055
6 0.9971884
6 0.99929523
6 0.55054516
3 0.53570664
6 0.75966537
6 0.9981766
6 0.9997725
7 0.8621095
5 0.73451245
5 0.9860423
7 0.7547408
7 0.9790258
7 0.9971559
7 0.982897
7 0.734072
5 0.99954337
5 0.87520856
5 0.7216308
5 0.54605377
5 0.84

5 0.87092805
5 0.87092805
5 0.96026707
5 0.89774805
5 0.9305049
5 0.873591
5 0.59500575
5 0.8201174
5 0.9186952
5 0.6470726
5 0.6648169
5 0.8552377
5 0.7721165
5 0.8199422
5 0.8847636
5 0.8011478
5 0.8865241
5 0.7910225
5 0.869323
5 0.90195197
5 0.5489779
5 0.90290874
5 0.81715024
5 0.88674146
5 0.88674146
5 0.90386176
5 0.795993
1 0.51351553
5 0.8397044
5 0.8597922
5 0.8546216
5 0.90329695
5 0.5908097
1 0.9147052
4 0.16484767
3 0.524084
3 0.524084
2 0.5206787
5 0.2052174
9 0.3102518
9 0.603963
6 0.4427509
11 0.9614767
11 0.69125795
11 0.9051556
11 0.97695816
11 0.89236885
11 0.89236885
5 0.55031735
5 0.89834625
7 0.9397515
5 0.84420574
5 0.63454175
5 0.8580603
5 0.829577
5 0.88860106
5 0.88929117
5 0.7858292
5 0.81293863
5 0.5643391
5 0.5643391
7 0.94151163
7 0.9911515
7 0.9927556
7 0.9952388
7 0.99140733
7 0.9791303
7 0.91729885
7 0.9427989
7 0.98023444
7 0.9979438
7 0.9986815
7 0.9986815
7 0.9994137
7 0.99951017
7 0.95086944
7 0.99819416
7 0.9813959
7 0.99807274
7 0.9988951
7 0.9994

9 0.9951443
9 0.9990459
9 0.99979466
9 0.9992719
9 0.99882525
9 0.9992605
9 0.9978155
9 0.9942415
9 0.9938666
9 0.9915725
9 0.99890935
9 0.999244
9 0.9913619
9 0.99985445
9 0.99908566
9 0.99911374
9 0.9837918
9 0.9985097
9 0.9925997
9 0.9993868
9 0.9900393
9 0.9910591
9 0.9922747
9 0.9770486
9 0.9946956
9 0.9860093
9 0.9862851
9 0.9855679
9 0.9876407
9 0.98781234
9 0.99366635
9 0.9969994
9 0.9976156
9 0.99926955
9 0.9946426
9 0.99250835
9 0.9919037
9 0.99912673
9 0.9876716
9 0.9901404
9 0.98561805
9 0.9925224
9 0.99595034
9 0.98850983
9 0.99178773
9 0.99308676
9 0.99253285
9 0.99129784
9 0.98948157
9 0.98774976
9 0.9877957
9 0.9857496
9 0.9908454
9 0.9883868
9 0.99342936
9 0.9855472
9 0.9884254
9 0.98916024
9 0.98816466
9 0.98720247
9 0.99070626
9 0.9842291
9 0.9872903
9 0.9922185
9 0.9863582
9 0.9877473
9 0.98051995
9 0.9891868
9 0.9865817
9 0.98581934
9 0.9891766
9 0.98107016
9 0.9880264
9 0.9948197
9 0.9886719
9 0.9844124
9 0.9715541
9 0.9999399
9 0.9974439
9 0.9762106
9 0.99713
9 0

...

5 0.7979196
5 0.8787928
5 0.9983846
7 0.98159057
5 0.7916963
5 0.7927622
5 0.51952
5 0.9888067
5 0.9047746
5 0.73348355
5 0.6601079
5 0.86730236
5 0.86730236
3 0.5217044
5 0.56064206
5 0.63088757
5 0.8911859
5 0.6986836
5 0.82099175
9 0.9999118
6 0.6793263
5 0.9867296
5 0.5953858
6 0.99778986
6 0.99778986
6 0.9813861
9 0.99946874
9 0.91446245
9 0.9936574
9 0.96786344
9 0.87858945
9 0.7884617
9 0.94897664
9 0.522571
5 0.67175764
9 0.9997571
9 0.9997571
9 0.9829695
9 0.9930809
9 0.81758624
5 0.82609457
9 0.9854806
9 0.9980842
9 0.9656298
5 0.88920623
9 0.9999949
5 0.99983907
5 0.9962159
5 0.84253085
9 0.9687242
5 0.98425657
9 1.0
9 0.9894706
9 0.99983597
9 0.9908428
9 0.96801734
9 0.97275835
9 0.9898567
9 0.9982961
9 0.9305814
9 0.9751912
9 0.94364196
5 0.99992955
7 0.8917453
3 0.7396801
5 0.7127655
9 0.9995864
5 0.95889455
7 0.9780853
5 0.88516885
5 0.996929
5 0.58677214
5 0.9822739
5 0.99450976
5 0.9934144
5 0.65886194
5 0.99496955
5 0.99999976
5 0.9802663
5 0.9802663
5 0.98992443
5 0.

9 0.9870717
9 0.9802172
9 0.9771821
9 0.97984743
9 0.9966857
9 0.9820463
9 0.99536186
9 0.99249744
9 0.95247
9 0.94432795
9 0.9928766
9 0.98303163
9 0.9931162
9 0.95874625
9 0.94342023
9 0.98959494
9 0.9870073
9 0.96209055
9 0.98885494
9 0.9885251
9 0.9984257
9 0.9865877
9 0.9856464
9 0.9947701
9 0.98583883
9 0.99689746
9 0.98911744
9 0.9937994
9 0.9974553
9 0.9974553
9 0.9944477
9 0.9831807
9 0.98796767
9 0.9891629
9 0.9973314
9 0.9967686
9 0.97584003
9 0.984259
9 0.9725566
9 0.9060002
9 0.90512615
9 0.90512615
9 0.977945
9 0.8888688
9 0.95401585
9 0.9495207
9 0.9634217
9 0.90710074
9 0.9523509
9 0.96914154
9 0.80760664
9 0.8947938
9 0.8947938
9 0.9712365
9 0.9499238
9 0.87267566
9 0.9344963
9 0.94547117
9 0.95344347
9 0.9266459
9 0.9732118
9 0.9352038
9 0.9069186
9 0.8573549
9 0.8573549
9 0.7992238
9 0.9033595
2 0.4152187
9 0.94173986
9 0.72693956
6 0.632876
9 0.90991193
9 0.8678498
9 0.9052429
9 0.6912035
9 0.7207713
9 0.90424687
9 0.9619718
9 0.9730546
9 0.9659831
9 0.8589802
9 0.8

8 0.6733925
1 0.48140863
11 0.61227924
8 0.74882334
11 0.43310088
8 0.77351743
3 0.57619584
3 0.76844764
1 0.38577262
11 0.61056453
11 0.59681404
8 0.4759048
11 0.99254066
11 0.618285
3 0.94863695
9 0.9999989
1 0.9993414
6 0.6352112
6 0.33980107
3 0.65009886
6 0.48231137
6 0.97783077
9 0.9998995
6 0.98305327
7 0.67686355
7 0.62650305
5 0.6128435
6 0.7770672
6 0.9064631
6 0.896181
6 0.98015255
6 0.6576804
6 0.99961615
6 0.99836046
5 0.9958264
9 0.88050437
5 0.5745624
5 0.99999964
5 0.9999995
5 0.99771583
5 0.9469063
5 0.99952245
5 0.9925417
9 0.62706685
11 0.9068599
5 0.9923908
5 0.9975745
5 0.99881494
1 0.99999654
1 0.95293015
5 0.7663682
5 0.5368501
5 0.9982449
5 0.9754734
5 0.9579225
11 0.89283526
5 0.99952126
9 0.99769586
9 0.54301995
9 0.99999774
9 0.9999988
9 0.99997735
9 1.0
9 0.99944276
9 0.99994457
1 0.99239457
9 0.99924976
9 0.99941885
9 0.9987863
9 0.9997912
9 0.94437456
9 0.98526984
9 0.97564924
9 0.9875278
9 0.99911326
9 0.999979
9 0.9999473
9 0.9999994
9 0.99904555
9 0.999

9 0.5241342
5 0.7654956
5 0.7654956
10 0.6305794
9 0.88893116
9 0.71037394
5 0.42088214
9 0.8764432
6 0.67958176
1 0.99999523
6 0.4628757
11 0.7152866
9 0.71835196
10 0.995242
10 0.995242
5 0.39979485
1 0.99999845
9 0.696457
11 0.54421586
11 0.8489044
9 0.9999999
1 0.9882117
9 0.98695683
1 0.9936046
9 0.99121094
9 0.59254026
9 0.59254026
5 0.879439
9 0.7901542
9 0.62103564
6 0.90136963
9 0.986262
11 0.36008164
9 0.60464436
1 0.9999682
6 0.76515734
1 0.9764763
9 0.99918026
10 0.48947936
6 0.9302064
9 0.5133961
8 0.5469487
6 0.62061816
6 0.92744905
1 0.9533097
9 0.9733324
9 0.8456592
1 0.9897766
9 0.8841204
9 0.72474074
9 0.89382696
9 0.8880506
1 0.8434879
9 0.99992263
9 0.99998987
9 0.99968207
9 0.9987602
9 0.9999887
9 0.96262777
9 0.90298504
1 0.7817303
1 0.7817303
1 0.9967601
11 0.9961688
11 0.99924254
11 0.7465139
2 0.72174364
9 0.8143912
8 0.7601745
1 0.9584221
10 0.80202
1 0.99999976
1 0.83714473
1 0.9932747
1 0.63193214
11 0.97347844
8 0.63443214
11 0.5983773
11 0.99986696
11 0.98

KeyboardInterrupt: 